# Miller Index

In [6]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink
import math
from ipywidgets import HBox, VBox, Button, Dropdown, ToggleButtons, Output, jslink, link
from numpy.linalg import norm
from sklearn.preprocessing import normalize
from traitlets import List


vertices = [
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
]

faces = [
    [0, 1, 3],
    [0, 3, 2],
    [0, 2, 4],
    [2, 6, 4],
    [0, 4, 1],
    [1, 4, 5],
    [2, 3, 6],
    [3, 7, 6],
    [1, 5, 3],
    [3, 5, 7],
    [4, 6, 5],
    [5, 6, 7]
]

surfaces = [
    [0, 0.5, 0.5],
    [1, 0.5, 0.5],
    [0.5, 0.5, 0],
    [0.5, 0.5, 1],
    [0.5, 0, 0.5],
    [0.5, 1, 0.5]
]

vertexcolors = ['#000000', '#0000ff', '#00ff00', '#ff0000',
                '#00ffff', '#ff00ff', '#ffff00', '#ffffff']

# Map the vertex colors into the 'color' slot of the faces
faces = [f + [None, [vertexcolors[i] for i in f], None] for f in faces]

# balls at vertex 
balls = [Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi,), 
            material=MeshLambertMaterial(color='red', opacity = 0.7, transparent = True),
            position=i) for i in vertices]

cball = Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi), 
            material=MeshLambertMaterial(color='gold'),
            position=[0.5, 0.5, 0.5])

sballs = [Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi), 
            material=MeshLambertMaterial(color='red', opacity = 0.7, transparent = True),
            position=i) for i in surfaces]

# Create the geometry:
cubeGeometry = Geometry(vertices=vertices,
    faces=faces,
    colors=vertexcolors,)
# Calculate normals per face, for nice crisp edges:
cubeGeometry.exec_three_obj_method('computeFaceNormals')

box = BoxGeometry(1,1,1)
edgesGemoetry = EdgesGeometry(box);

# Create a mesh. Note that the material need to be told to use the vertex colors.
myobjectCube = Mesh(
    geometry=box,
    material= MeshPhongMaterial(color = 'blue', opacity = 0.4, transparent = True),
    position=[0.5, 0.5, 0.5],   # Center the cube
)


edges = LineSegments(
    geometry = edgesGemoetry,
    material = LineBasicMaterial(color = 'black', linewidth = 3),
    position = [0.5, 0.5, 0.5]
)

planeGeometry = Geometry(vertices = [[1, 0, 0], [0, 1, 0], [0, 0, 1]], faces=[[0, 1, 2, None, ['red'], None]])

planes = Mesh(
    geometry = planeGeometry,
    material = MeshBasicMaterial(color = 'blue', side = 'DoubleSide'),
    position = [0, 0, 0])

axesHelper = AxesHelper(1.2);

arrow = ArrowHelper([1/sqrt(3), 1/sqrt(3), 1/sqrt(3)], [0, 0, 0], 1.5, color = '#ff0000')

# Set up a scene and render it:
cCube = PerspectiveCamera(position=[3, 3, 3], fov=20,
                      children=[DirectionalLight(color='#ffffff', position=[-3, 5, 1], intensity=0.5)])



out = Output();

buttons_h = ToggleButtons(
    options = [0, 1, 2],
    value = 1,
    description = 'h:',
    disabled = False
)

buttons_k = ToggleButtons(
    options = [0, 1, 2],
    value = 1,
    description = 'k:',
    disabled = False
)

buttons_l = ToggleButtons(
    options = [0, 1, 2],
    value = 1,
    description = 'l:',
    disabled = False
)

def on_h_change(change):
    #planes.geometry.verticesNeedUpdate = True
    #planes.geometry.vertices = [[float(buttons_h.value), 0, 0], [0, 1, 0], [0, 0, 1]]  
    #planes.geometry.verticesNeedUpdate = True
    planes.geometry = Geometry(vertices = [[buttons_h.value, 0, 0], [0, buttons_k.value, 0], 
                                           [0, 0, buttons_l.value]], faces=[[0, 1, 2, None, ['red'], None]])
    #planes.geometry.verticesNeedUpdate = True
    #planes.geometry.dynamic = True
    #planes.geometry.exec_three_obj_method('update')
    #sceneCube.exec_three_obj_method('update')

def on_k_change(change):
    planes.geometry = Geometry(vertices = [[buttons_h.value, 0, 0], [0, buttons_k.value, 0], [0, 0, buttons_l.value]], 
                               faces=[[0, 1, 2, None, ['red'], None]])
    
def on_l_change(change):
    planes.geometry = Geometry(vertices = [[buttons_h.value, 0, 0], [0, buttons_k.value, 0], [0, 0, buttons_l.value]], 
                               faces=[[0, 1, 2, None, ['red'], None]])
    
buttons_h.observe(on_h_change, names = 'value')
buttons_k.observe(on_h_change, names = 'value')
buttons_l.observe(on_h_change, names = 'value')

sceneCube = Scene(children= [edges, planes, axesHelper, arrow, AmbientLight(color='#dddddd')])

rendererCube = Renderer(camera=cCube, background='black', background_opacity=1,
                        scene=sceneCube, controls=[OrbitControls(controlling=cCube)], 
                        width=500, height=500)

In [7]:
display(VBox([rendererCube, buttons_h, buttons_k, buttons_l]))